In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import tensorflow
import pandas as pd
import time
import numpy as np
import math


# tensorflow libraries
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, concatenate, Input, Flatten

from tensorflow.keras import layers
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
from tensorflow.keras import optimizers

# sklearn libraries are useful for preprocessing, performance measures, etc.
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/FYP/SG/training.csv')
val_df = pd.read_csv('/content/drive/MyDrive/FYP/SG/validation.csv')

lat = df.pop("lat_3414")
lon = df.pop('lon_3414')
df.pop("floor_id")
df.pop("timestamp")
df.pop("type")
df.pop("latitude")
df.pop("longitude")


val_lat = val_df.pop("latitude")
val_lon = val_df.pop('longitude')
val_df.pop("floor_id")
val_df.pop("timestamp")
val_df.pop("type")


0        truth
1       pseudo
2       pseudo
3       pseudo
4       pseudo
        ...   
692     pseudo
693     pseudo
694     pseudo
695     pseudo
696      truth
Name: type, Length: 697, dtype: object

In [5]:
from math import radians, cos, sin, asin, sqrt

def single_pt_haversine(lat, lng, degrees=True):
    """
    'Single-point' Haversine: Calculates the great circle distance
    between a point on Earth and the (0, 0) lat-long coordinate
    """
    r = 6371 # Earth's radius (km). Have r = 3956 if you want miles

    # Convert decimal degrees to radians
    if degrees:
        lat, lng = map(radians, [lat, lng])

    # 'Single-point' Haversine formula
    a = sin(lat/2)**2 + cos(lat) * sin(lng/2)**2
    d = 2 * r * asin(sqrt(a)) 

    return d
    

In [6]:
def dataframe_to_dataset(dataframe, label):
    dataframe = dataframe.copy()
    labels = label
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

In [7]:
labels = []
for i in range (len(lat)):
  labels.append(single_pt_haversine(lat[i],lon[i]))
  # temp = []
  # temp.append(lat[i])
  # temp.append(lon[i])
  # labels.append(temp)


val_labels = []
for i in range (len(val_lat)):
  val_labels.append(single_pt_haversine(val_lat[i],val_lon[i]))
  # temp = []
  # temp.append(val_lat[i])
  # temp.append(val_lon[i])
  # val_labels.append(temp)

print(val_labels[2])


#print(val_df.iloc[[2]])

11547.371913868174


In [10]:
train_ds = dataframe_to_dataset(df, labels)

In [ ]:

# report which features were selected by RFE
from sklearn.feature_selection import RFE
# define dataset
X, y = df, labels
# define RFE
rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=400)
# fit RFE
rfe.fit(X, y)
# summarize all features
for i in range(X.shape[1]):
	print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

In [ ]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

In [ ]:
selected = [] 
for i in range(X.shape[1]):
	#print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
  selected.append(rfe.support_[i])

print((selected[52]))

index_pos_list = get_index_positions(selected, False)
print((index_pos_list))



True
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 102, 103, 111, 122, 123, 127, 130, 134, 138, 142, 144, 146, 147, 155, 157, 158, 162, 163, 167, 169, 170, 171, 172, 177, 178, 179, 180, 183, 184, 186, 189, 198, 199, 200, 201, 211, 212, 216, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 232, 234, 238, 239, 240, 242, 243, 244, 246, 248, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 267, 268, 270, 273, 274, 275, 276, 277, 278, 279, 280, 283, 284, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 345, 346, 347, 348, 349, 350, 352, 353, 355, 357, 358, 359, 360, 361, 362, 363, 364, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407,

In [ ]:
df_tosave = pd.read_csv('/content/drive/MyDrive/FYP/SG/training.csv')
val_df_tosave = pd.read_csv('/content/drive/MyDrive/FYP/SG/validation.csv')

In [ ]:
df_tosave.drop(df_tosave.columns[index_pos_list], axis = 1, inplace = True) 
 

IndexError: ignored

In [ ]:
df_tosave.head()

,WAP10,WAP11,WAP12,WAP13,WAP14,WAP15,WAP16,WAP17,WAP18,WAP19,WAP20,WAP21,WAP22,WAP23,WAP24,WAP25,WAP26,WAP27,WAP28,WAP29,WAP30,WAP31,WAP32,WAP33,WAP34,WAP35,WAP36,WAP37,WAP38,WAP39,WAP40,WAP41,WAP42,WAP43,WAP44,WAP45,WAP46,WAP47,WAP48,WAP49,...,WAP778,WAP779,WAP780,WAP781,WAP782,WAP783,WAP784,WAP785,WAP789,WAP790,WAP792,WAP795,WAP797,WAP798,WAP799,WAP800,WAP801,WAP802,WAP803,WAP805,WAP806,WAP807,WAP808,WAP809,WAP822,WAP825,WAP826,WAP827,WAP828,WAP829,WAP830,WAP840,WAP846,floor_id,timestamp,longitude,latitude,type,lon_3414,lat_3414
10,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-84,-84,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,2,1.630000e+12,103.851560,1.290506,pseudo,30030.136347,30323.086077
11,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-83,-110,-81,-82,-80,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,2,1.630000e+12,103.851556,1.290499,pseudo,30029.657800,30322.305525
12,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-85,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,2,1.630000e+12,103.851554,1.290495,truth,30029.424091,30321.915195
13,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-79,-74,-74,-75,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,2,1.630000e+12,103.851547,1.290499,pseudo,30028.633918,30322.372748
14,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-89,-85,-85,-85,-84,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,2,1.630000e+12,103.851540,1.290503,pseudo,30027.843745,30322.830411


In [ ]:
df_tosave.to_csv("test.csv",index=False)

In [ ]:
val_df_tosave.drop(val_df_tosave.columns[index_pos_list], axis = 1, inplace = True)
val_df_tosave.head()


,WAP10,WAP11,WAP12,WAP13,WAP14,WAP15,WAP16,WAP17,WAP18,WAP19,WAP20,WAP21,WAP22,WAP23,WAP24,WAP25,WAP26,WAP27,WAP28,WAP29,WAP30,WAP31,WAP32,WAP33,WAP34,WAP35,WAP36,WAP37,WAP38,WAP39,WAP40,WAP41,WAP42,WAP43,WAP44,WAP45,WAP46,WAP47,WAP48,WAP49,...,WAP778,WAP779,WAP780,WAP781,WAP782,WAP783,WAP784,WAP785,WAP789,WAP790,WAP792,WAP795,WAP797,WAP798,WAP799,WAP800,WAP801,WAP802,WAP803,WAP805,WAP806,WAP807,WAP808,WAP809,WAP822,WAP825,WAP826,WAP827,WAP828,WAP829,WAP830,WAP840,WAP846,floor_id,timestamp,longitude,latitude,type,lon_3414,lat_3414
0,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-76,-77,-78,-70,-71,-70,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,2,1.630000e+12,103.851602,1.290576,truth,30034.832791,30330.891589
1,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-78,-77,-78,-72,-72,-71,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,2,1.630000e+12,103.851598,1.290569,pseudo,30034.387630,30330.143547
2,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-83,-110,-110,-75,-75,-75,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,2,1.630000e+12,103.851594,1.290563,pseudo,30033.931341,30329.395505
3,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-74,-74,-74,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,2,1.630000e+12,103.851590,1.290556,pseudo,30033.486180,30328.647463
4,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-81,-110,-110,-77,-78,-79,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,2,1.630000e+12,103.851586,1.290549,pseudo,30033.029891,30327.899420


In [ ]:
val_df_tosave.to_csv("valtest.csv",index=False)

# IGNORE

In [ ]:

rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=5)
model = RandomForestRegressor()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# fit the model on all available data
pipeline.fit(df, labels) 
data = val_df.iloc[[2]]
yhat = pipeline.predict(data)
print(yhat)

[11547.37000933]


In [ ]:
print(val_labels[2])


11547.371913868174


In [ ]:
from matplotlib import pyplot
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# get a list of models to evaluate
def get_models():
  models = dict()
  for i in range(2, 10):
    rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=i)
    model = RandomForestRegressor()
    models[str(i)] = Pipeline(steps=[('s',rfe),('m',model)])
    return models

def get_one_model():
  rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=5)
  model = RandomForestRegressor()
  pipeline = Pipeline(steps=[('s',rfe),('m',model)])
  return pipeline 
  


# define dataset
X, y = df, labels
# get the models to evaluate
models = get_one_model()
# evaluate the models and store results
results, names = list(), list()
train_1 = model.fit(X,y) 
 
  


AttributeError: ignored

In [ ]:
data = val_df.iloc[[11]]
yhat = train_1.predict(data)
print(yhat)
print(val_labels[11])



In [ ]:
train_1.support_


AttributeError: ignored